In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import os

# Custom Dataset 클래스
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        
        for label in ['real', 'fake']:
            folder_path = os.path.join(root_dir, label)
            for img_name in os.listdir(folder_path):
                if img_name.endswith('.jpg'):
                    self.image_paths.append(os.path.join(folder_path, img_name))
                    self.labels.append(0 if label == 'real' else 1)
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# 이미지 전처리
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# DataLoader 설정
train_dataset = CustomDataset(root_dir='DataSet3/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = CustomDataset(root_dir='DataSet3/test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# CNN 모델 정의
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 32 * 32, 512)
        self.fc2 = nn.Linear(512, 2)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 32 * 32)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# 손실 함수 및 옵티마이저 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNNModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.2f}%")
    
    save_dir = 'models'  # 모델 저장할 폴더
    os.makedirs(save_dir, exist_ok=True)  # 폴더가 없으면 생성
    torch.save(model.state_dict(), os.path.join(save_dir, f"cnn_model_epoch_{epoch+1}.pth"))

# 모델 평가
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")


# 최종 모델 저장 (학습이 끝난 후)
final_model_path = 'models/cnn_model_final.pth'
torch.save(model.state_dict(), final_model_path)
print(f"Final model saved to {final_model_path}")

Epoch [1/10], Loss: 0.4534, Accuracy: 77.62%
Epoch [2/10], Loss: 0.2526, Accuracy: 89.22%
Epoch [3/10], Loss: 0.1775, Accuracy: 92.57%
Epoch [4/10], Loss: 0.1312, Accuracy: 94.72%
Epoch [5/10], Loss: 0.0949, Accuracy: 96.19%
Epoch [6/10], Loss: 0.0736, Accuracy: 97.20%
Epoch [7/10], Loss: 0.0617, Accuracy: 97.62%
Epoch [8/10], Loss: 0.0474, Accuracy: 98.19%
Epoch [9/10], Loss: 0.0481, Accuracy: 98.26%
Epoch [10/10], Loss: 0.0407, Accuracy: 98.58%
Test Accuracy: 59.84%
Final model saved to models/cnn_model_final.pth
